Import required modules

In [10]:
import sys
import numpy as np
import random

Set default arguments in case value is not provided from console

type values - {"isolated", "partition"}

In [11]:
n = 3
m = 20
type = None

Uncoment this code block in case you are providing arguments from console

In [12]:
# args = len(sys.argv)

# if args > 1:
# 	n = int(sys.argv[1])
# if args > 2:
# 	m = int(sys.argv[2])
# if args > 3:
# 	type = sys.argv[3]

Setup some initial parameters for hashgraph

In [13]:
spare = np.array(range(1000,9999)) # spare ids left for events
curr_event = [None]*n              # id of last event by a node
curr_time = [0]*n                  # timer for each node
curr_isolated = [False]*n          # keep track of currently isolated nodes
partition_flag = False             # flag for if partition is present
hashgraph = []                     # list of events in hashgraph

curr_m = 0                         # current number of events created

Setup different parameters for isolated type

In [14]:
if type=='isolated' and n>1:
	isolated_num = random.randint(1,n-1)
	isolated_nodes = np.array([0]*n)
	isolated_till = np.array([0]*n)
	isolated_after = np.array([0]*n)
	for i in range(isolated_num):
		isolated_node = random.choice(*np.where(isolated_nodes == 0))
		isolated_nodes[isolated_node] = 1
		isolated_till[isolated_node] = random.randint(int(m/4)+1, m)
		isolated_after[isolated_node] = random.randint(0,int(m/4))

Setup different parameters for partition type

In [15]:
if type=='partition' and n>1:
	partition = np.array([0]*n)
	for i in range(int(n/2)):
		node = random.choice(*np.where(partition==0))
		partition[node]=1
	partition_after = random.randint(0,int(m/4))
	partition_till = random.randint(int(m/2),int(3*m/4))

Perform the gossip protocol between the nodes

In [16]:
while(curr_m<m):
		for node in range(0,n):
			create = random.choice([True, False])     # randomly occurance to choose to create event
			if create and not curr_isolated[node]:
        # span is the list of nodes to select randomly from
				if curr_event[node] is None:
					span = np.array([node])
				else:
					span = np.array([x for x in range(n) if x != node and not curr_isolated[x]])
					if partition_flag:
						span = np.array([x for x in range(n) if x != node and not curr_isolated[x] and partition[x]==partition[node]])
				gossip_node= random.choice(span)                # node chosen to gossip to
				event_id = random.choice(range(0,spare.size))   # id of the event 
				timestamp = max(curr_time[node], curr_time[gossip_node]) + 1    # timestamp for event
				event = [gossip_node+1,spare[event_id],curr_event[node],curr_event[gossip_node],timestamp]
				curr_time[gossip_node] = timestamp   # update current time for gossip node  
				curr_event[gossip_node] = spare[event_id]   # update latest event for gossip node
				hashgraph.append(event)
				spare = np.delete(spare,event_id)     # remove the id used
				curr_m += 1     # increase number of events created
      # isolate or un-isolate nodes according to the parameters
			if type=='isolated':
				for i in range(n):
					if(m==isolated_after[i]):
						curr_isolated[i] = True
					if(m==isolated_till[i]):
						curr_isolated[i] = False
      # create or remove partition according to the parameters
			if type=='partition':
				if(m==partition_after):
					partition_flag = True
				if(m==partition_till):
					partition_flag = False
			curr_time[node] += 1    # increase local clock

Comment this out in case you want output in a file

In [17]:
# fout = 'graph.txt'
# sys.stdout = open(fout, 'w')


Output the events

In [18]:
print(n)
for i in hashgraph:
	print(i[0], i[1], "NULL" if i[2] is None else i[2], "NULL" if i[3] is None else i[3], i[4])
print("Done")

3
3 4133 NULL NULL 2
1 2524 NULL NULL 3
2 7540 4133 NULL 5
3 7619 2524 4133 6
1 9895 7540 2524 7
1 4218 7540 9895 9
1 3148 7619 4218 10
2 4183 3148 7540 11
1 9704 4183 3148 12
3 5060 9704 7619 13
3 2360 4183 5060 14
3 8078 9704 2360 16
3 5931 4183 8078 17
2 1925 5931 4183 19
3 8885 1925 5931 20
3 6438 1925 8885 22
2 9794 6438 1925 24
2 5694 9704 9794 25
3 4631 5694 6438 26
2 9443 4631 5694 27
Done
